In [0]:
!pip install transformers

     |████████████████████████████████| 368kB 9.2MB/s 
     |████████████████████████████████| 645kB 67.9MB/s 
     |████████████████████████████████| 1.0MB 56.2MB/s 
     |████████████████████████████████| 860kB 74.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=9375db6b0c7b19355873311a1f57f5cd1a6d16dde7d820ad9925cc6f9e6a9db2
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
!pip install -U sentence-transformers

     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 163kB 21.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.3-cp36-none-any.whl size=57413 sha256=17cb18ca192fafc7dd6dcff567850a7a9f7598747e683a27cc7711d9e0d95e99
  Stored in directory: /root/.cache/pip/wheels/20/89/88/95d8a3d4034bb77f52a6f66efdbfc623240221183dc0001c31
Successfully built sentence-transformers


In [0]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime


#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
sts_reader = STSDataReader('stsbenchmark', normalize_scores=True)

# Use BERT for mapping tokens to embeddings
word_embedding_model = models.BERT('bert-base-uncased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


2019-11-28 08:19:18 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-11-28 08:19:18 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2019-11-28 08:19:19 - loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at /root/.cache/torch/pytorch_tran

In [0]:
import urllib.request
import zipfile
import os
#folder_path = os.path.dirname(os.path.realpath(__file__))
folder_path = "./"
print('Beginning download of datasets')

datasets = ['stsbenchmark.zip']
server = "https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/"

for dataset in datasets:
    print("Download", dataset)
    url = server+dataset
    dataset_path = os.path.join(folder_path, dataset)
    urllib.request.urlretrieve(url, dataset_path)

    print("Extract", dataset)
    with zipfile.ZipFile(dataset_path, "r") as zip_ref:
        zip_ref.extractall(folder_path)
    os.remove(dataset_path)


Beginning download of datasets
Download AllNLI.zip
Extract AllNLI.zip
Download stsbenchmark.zip
Extract stsbenchmark.zip
Download wikipedia-sections-triplets.zip
Extract wikipedia-sections-triplets.zip
All datasets downloaded and extracted


In [0]:

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_data = SentencesDataset(sts_reader.get_examples('sts-train.csv'), model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('sts-dev.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_data)*num_epochs/train_batch_size*0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_data = SentencesDataset(examples=sts_reader.get_examples("sts-test.csv"), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)
model.evaluate(evaluator)

Convert dataset:   2%|▏         | 98/5749 [00:00<00:06, 844.45it/s]

2019-11-28 08:19:27 - Read STSbenchmark train dataset


Convert dataset:  26%|██▌       | 391/1500 [00:00<00:00, 3899.20it/s]

2019-11-28 08:19:29 - Num sentences: 5749
2019-11-28 08:19:29 - Sentences 0 longer than max_seqence_length: 0
2019-11-28 08:19:29 - Sentences 1 longer than max_seqence_length: 0
2019-11-28 08:19:29 - Read STSbenchmark dev dataset


Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

...

Convert Evaluating:   0%|          | 0/87 [00:00<?, ?it/s]

2019-11-28 11:58:21 - Num sentences: 1379
2019-11-28 11:58:21 - Sentences 0 longer than max_seqence_length: 0
2019-11-28 11:58:21 - Sentences 1 longer than max_seqence_length: 0
2019-11-28 11:58:21 - Evaluation the model on  dataset:


Convert Evaluating: 100%|██████████| 87/87 [01:58<00:00,  1.13it/s]

2019-11-28 12:00:19 - Cosine-Similarity :	Pearson: 0.8507	Spearman: 0.8497
2019-11-28 12:00:19 - Manhattan-Distance:	Pearson: 0.8381	Spearman: 0.8384
2019-11-28 12:00:19 - Euclidean-Distance:	Pearson: 0.8390	Spearman: 0.8392
2019-11-28 12:00:19 - Dot-Product-Similarity:	Pearson: 0.7777	Spearman: 0.7690


0.8497360050809678

In [0]:
!zip -r output.zip output

  adding: output/ (stored 0%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/ (stored 0%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/config.json (stored 0%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/1_Pooling/ (stored 0%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/1_Pooling/config.json (deflated 47%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/similarity_evaluation_results.csv (deflated 50%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/modules.json (deflated 51%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/0_BERT/ (stored 0%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/0_BERT/config.json (deflated 51%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/0_BERT/vocab.txt (deflated 53%)
  adding: output/training_stsbenchmark_bert-2019-11-28_08-19-18/0_BERT/pytorch_model.bin (deflated 7%)
  adding: output/training_stsbe

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
root_dir = "/content/gdrive/My Drive/"

In [0]:
!cp output.zip '/content/gdrive/My Drive/bert_finetuned_stsb.zip'